# Environment variables setup

In [2]:
from secret_key import api_key
import os
os.environ["OPENAI_API_KEY"] = api_key

# Using Naive Prompts

In [3]:
from langchain.llms import OpenAI

In [7]:
# naive prompting
llm = OpenAI(temperature=0.6)
name = llm("I want to open an Indian dress shop. Suggest a fancy name for this")
name

' shop\n\n"The Indian Closet"'

# Working with Prompt Templates

In [8]:
# creating prompt template
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables = ["origin"],
    template = "I want to open a dress boutique for {origin} clothes. Suggest a fancy name for this."
)

# prompt generated using template
prompt_template.format(origin = "Arabic")

'I want to open a dress boutique for Arabic clothes. Suggest a fancy name for this.'

# Using LLMChains

In [9]:
from langchain.chains import LLMChain

# creating chain = llm + prompt_template
chain = LLMChain(llm=llm, prompt = prompt_template)

# inference with LLMChain, by passing input variables
chain.run("American")

'\n\nDressed to Impress Boutique'

# Sequential Chaining

In [10]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables = ["origin"],
    template="I want to open a dress boutique for {origin} clothes. Suggest a fancy name for this."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables=["boutique_name"],
    template="""Suggest some dress items for {boutique_name}.
                Return it as comma separated values"""
)

items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [11]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, items_chain])
response = chain.run("Indian")
response

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).


'\n\nSilk sarees, Benarasi sarees, Chanderi sarees, Kalamkari sarees, Kanjeevaram sarees, Lehenga sarees, Patiala salwar suits, Anarkali suits, Bandhani sarees, Bollywood sarees.'

With SimpleSequentialChains, recieve only a single output

# Using SequentialChains

In [13]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables=["origin"], 
    template="I want to open a dress boutique for {origin} dresses. Suggest a fancy name for this."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="boutique_name")

prompt_template_items = PromptTemplate(
    input_variables=["boutique_name", "origin"],
    template="""Suggest some {origin} dress items for {boutique_name}.
                Return it as comma separated values"""
)

items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="dress_items")

In [14]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, items_chain],
    input_variables = ["origin"],
    output_variables = ["boutique_name", "dress_items"]
)

chain({"origin": "Arabic"})

{'origin': 'Arabic',
 'boutique_name': '\n\nAl-Firdaws Couture Boutique',
 'dress_items': '\n\nAbaya, Hijab, Jalabiya, Kaftan, Sheyla, Bisht, Dishdasha, Thawb, Sirwal, Sartoura'}